Наши данные

In [1]:
import pandas as pd
import re

pth = '/home/sondors/Documents/price/BERT_NER/csv/Kazakov_241123/Cordiant_character.xlsx'   
df = pd.read_excel(pth, dtype=str)
df
# print(df.keys())

,PRICE_FRNO,PRICE_FNAM,PRICE_CAID,PRICE_CNAM,PRICE_TMNO,PRICE_TMRK,Наименование,Модель,Ширина шины 135-355,Высота профиля 25-100,...,PRICE_RSLT,kpl,ppl,PRICE_IMGN,PRICE_LOCD,lineNumber,PRICE_SALES,PRICE_PLID,Unnamed: 34,Unnamed: 35
0,96794,NaN,8101,NaN,9266,NaN,15/205/70 Cordiant All Terrain 100H,All-Terrain,205,70,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Cordiant,https://avatars.mds.yandex.net/get-mpic/374158...,"[8101, 821601]",2023-07-29 18:50:34,NaN,1705350501
1,96794,NaN,8101,NaN,9266,NaN,205/70 R15 Cordiant All Terrain 100H,All-Terrain,205,70,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Cordiant,https://avatars.mds.yandex.net/get-mpic/374158...,[8101],2023-06-05 20:59:59,NaN,1745121896
2,96794,NaN,8101,NaN,9266,NaN,205/70R15 Cordiant All Terrain OA-1 100H,All-Terrain,205,70,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Cordiant,https://avatars.mds.yandex.net/get-mpic/751999...,"[8101, 8104]",2023-07-29 18:50:34,NaN,1749848907
3,96794,NaN,8101,NaN,9266,NaN,Cordiant (Кордиант) all terrain 205/70R15 100H,All-Terrain,205,70,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Cordiant,https://avatars.mds.yandex.net/get-mpic/374158...,[8101],2023-07-16 08:13:02,NaN,1745711016
4,96794,NaN,8101,NaN,9266,NaN,Cordiant 205/70 R15 100H All Terrain OA-1,All-Terrain,205,70,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Cordiant,https://avatars.mds.yandex.net/get-mpic/374158...,"[8101, 730701]",2023-06-22 18:45:02,NaN,1749958452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4943,92652,NaN,8101,NaN,9266,NaN,Cordiant Шина Winter Drive 215/70 R16 100T,Winter Drive,215,70,...,NaN,NaN,Auto,NaN,Cordiant,https://kazan.kolesa-darom.ru/upload/iblock/68...,[8101],2023-02-09 17:23:07,NaN,1666127286
4944,99886,NaN,8101,NaN,9266,NaN,R16 215/70 100T Cordiant Winter Drive,Winter Drive,215,70,...,494490412,NaN,Auto,ШИНЫ :: Легковые,CORDIANT,https://yourtyres.ru/image/cache/catalog/YML02...,[8101],2023-04-11 23:23:07,NaN,1690196220
4945,93135,NaN,8101,NaN,9266,NaN,R16 215/70 Cordiant Winter Drive PW-1 100T,Winter Drive,215,70,...,NaN,NaN,Auto,Автотовары :: Шины и диски :: Шины,Cordiant,https://main-cdn.sbermegamarket.ru/hlr-system/...,[8101],2023-01-27 12:06:35,NaN,1637449289
4946,95652,NaN,8101,NaN,9266,NaN,Cordiant Winter Drive 205/55 R16 94T Run Flat,Winter Drive RunFlat,205,55,...,366617336,NaN,Auto,Зимние шины,Cordiant,https://www.citytire.ru/loads/catalogue/model/...,[8101],2023-02-20 19:35:16,"Наличные, Visa/Mastercard/МИР, б/н расчет",1676265135


In [2]:
df.PRICE_LOCD.value_counts()

Cordiant               4308
CORDIANT                562
Cordiant(Кордиант)       15
Cordiant (Кордиант)       4
cordiant                  4
Кордиант                  2
втехнике.рф               2
Cordiant sport-3          1
Name: PRICE_LOCD, dtype: int64

In [3]:
def process_text(input_text):
    def separate_letters_and_numbers(input_text):
        # Используем регулярное выражение для поиска сочетаний букв и цифр
        pattern = re.compile(r'(\D+|\d+)')
        
        # Используем findall для нахождения всех сочетаний
        matches = pattern.findall(input_text)
        # Возвращаем строку с пробелами между буквами и цифрами
        return ' '.join(matches)

    processed_text = separate_letters_and_numbers(input_text)    
    processed_text = processed_text.replace("|", " | ")
    processed_text = processed_text.replace("(", " ( ")
    processed_text = processed_text.replace(")", " ) ")
    processed_text = processed_text.replace("[", " [ ")
    processed_text = processed_text.replace("]", " ] ")
    # Убираем повторяющиеся пробелы
    processed_text = re.sub(r'  +', ' ', processed_text)
    return processed_text

def create_bio_tags(row):
    name = str(row['offer'])
    width = str(row['width'])
    height = str(row['height'])
    radius = str(row['radius'])
    brand = row['brand']
    line = str(row['line'])
    v_ind = str(row['v_ind'])

    # Создаем пустой список для хранения тегов
    tags = ['O'] * len(name.split())
    Begin_line = True
    Begin_v_ind = True
    Begin_radius = True
    # Проверяем каждое слово в наименовании и устанавливаем соответствующий тег
    for i, word in enumerate(name.split()):
        if word == '/':
            continue
        
        #   width
        if word == width:
            tags[i] = 'B-width'
            width = width.replace(word,"")

        #   height
        elif word == height:
            tags[i] = 'B-height'
            height = height.replace(word,"")

        #   radius    
        elif word.upper() in radius.upper():
            if radius.replace(" ","") == "":
                continue

            if Begin_radius:
                tags[i] = 'B-radius'
                Begin_radius = False
            else:
                tags[i] = 'I-radius'
            radius = radius.replace(word.lower(),"").replace(word.upper(),"")

        #   brand
        elif word in brand:
            tags[i] = 'B-brand'

        #   v_ind
        elif word.upper() in v_ind.upper():
            if v_ind.replace(" ","") == "":
                continue

            if Begin_v_ind:
                tags[i] = 'B-v_ind'
                Begin_v_ind = False
            else:
                tags[i] = 'I-v_ind'  
            v_ind = v_ind.replace(word.lower(),"").replace(word.upper(),"")  

        #   line
        elif word.upper() in line.upper() or re.sub(r"[^a-zA-Zа-яА-Я]","", word.upper().replace("СПОРТ", "SPORT")) in line.upper():
            if len(re.sub(r"[^a-zA-Zа-яА-Я]","", word.upper().replace("СПОРТ", "SPORT")))<3 and Begin_line:
                continue
            if line.replace(" ","") == "":
                continue
            
            if Begin_line:
                tags[i] = 'B-line'
                Begin_line = False
            else:
                tags[i] = 'I-line'
            line = line.replace(word,"")

    # Собираем теги в строку
    return ','.join(tags)

def change_columns(df):   
    new_df = pd.DataFrame()

    df = df.rename(columns={'Наименование': 'offer'})
    df = df[df['offer'].notna()]

    new_df['offer'] = df['offer']
    new_df['width'] = df['Ширина шины 135-355']
    new_df['height'] = df['Высота профиля 25-100']
    new_df['radius'] = df['Диаметр 12-24 R']
    new_df['v_ind'] = df['Индекснагрузкиискорости54-200+буква']
    new_df['brand'] = df['PRICE_LOCD']
    new_df['line'] = df['Модель']

    new_df['offer'] = new_df['offer'].apply(process_text)
    new_df['brand'] = new_df['offer'].apply(extract_Cordiant)
    new_df['BIO_Tags'] = new_df.apply(create_bio_tags, axis=1)
    return new_df

def extract_Cordiant(text):
    brands = re.findall(r'Cordiant|CORDIANT|Кордиант|cordiant|КОРДИАНТ|Кордиан|КОРДИАН', text)
    return brands

new_df = change_columns(df)
new_df

,offer,width,height,radius,v_ind,brand,line,BIO_Tags
0,15 / 205 / 70 Cordiant All Terrain 100 H,205,70,R15,100H,[Cordiant],All-Terrain,"B-radius,O,B-width,O,B-height,B-brand,B-line,I..."
1,205 / 70 R 15 Cordiant All Terrain 100 H,205,70,R15,100H,[Cordiant],All-Terrain,"B-width,O,B-height,B-radius,I-radius,B-brand,B..."
2,205 / 70 R 15 Cordiant All Terrain OA- 1 100 H,205,70,R15,100H,[Cordiant],All-Terrain,"B-width,O,B-height,B-radius,I-radius,B-brand,B..."
3,Cordiant ( Кордиант ) all terrain 205 / 70 R 1...,205,70,R15,100H,"[Cordiant, Кордиант]",All-Terrain,"B-brand,O,B-brand,O,B-line,I-line,B-width,O,B-..."
4,Cordiant 205 / 70 R 15 100 H All Terrain OA- 1,205,70,R15,100H,[Cordiant],All-Terrain,"B-brand,B-width,O,B-height,B-radius,I-radius,B..."
...,...,...,...,...,...,...,...,...
4943,Cordiant Шина Winter Drive 215 / 70 R 16 100 T,215,70,R16,100T,[Cordiant],Winter Drive,"B-brand,O,B-line,I-line,B-width,O,B-height,B-r..."
4944,R 16 215 / 70 100 T Cordiant Winter Drive,215,70,R16,100T,[Cordiant],Winter Drive,"B-radius,I-radius,B-width,O,B-height,B-v_ind,I..."
4945,R 16 215 / 70 Cordiant Winter Drive PW- 1 100 T,215,70,R16,100T,[Cordiant],Winter Drive,"B-radius,I-radius,B-width,O,B-height,B-brand,B..."
4946,Cordiant Winter Drive 205 / 55 R 16 94 T Run Flat,205,55,R16,94T,[Cordiant],Winter Drive RunFlat,"B-brand,B-line,I-line,B-width,O,B-height,B-rad..."


In [4]:
new_df.to_csv('/home/sondors/Documents/price/BERT_NER/csv/Kazakov_241123/разметка/Cordiant.csv', sep=';',index=False)

In [5]:
index = 3
example = dict(new_df.iloc[index])
print(example['offer'],'\n')
offer = example['offer'].split()
BIO_Tags = example['BIO_Tags'].split(',')

for i in range(len(offer)):
    print(f"{offer[i]}: {BIO_Tags[i]}")

Cordiant ( Кордиант ) all terrain 205 / 70 R 15 100 H 

Cordiant: B-brand
(: O
Кордиант: B-brand
): O
all: B-line
terrain: I-line
205: B-width
/: O
70: B-height
R: B-radius
15: I-radius
100: B-v_ind
H: I-v_ind


In [6]:
example

{'offer': 'Cordiant ( Кордиант ) all terrain 205 / 70 R 15 100 H',
 'width': '205',
 'height': '70',
 'radius': 'R15',
 'v_ind': '100H',
 'brand': ['Cordiant', 'Кордиант'],
 'line': 'All-Terrain',
 'BIO_Tags': 'B-brand,O,B-brand,O,B-line,I-line,B-width,O,B-height,B-radius,I-radius,B-v_ind,I-v_ind'}

In [7]:
example

{'offer': 'Cordiant ( Кордиант ) all terrain 205 / 70 R 15 100 H',
 'width': '205',
 'height': '70',
 'radius': 'R15',
 'v_ind': '100H',
 'brand': ['Cordiant', 'Кордиант'],
 'line': 'All-Terrain',
 'BIO_Tags': 'B-brand,O,B-brand,O,B-line,I-line,B-width,O,B-height,B-radius,I-radius,B-v_ind,I-v_ind'}

In [8]:
label2id = {'B-width': 1,
            'B-height': 2, 
            'B-radius': 3, 
            'I-radius': 4,
            'B-brand': 5, 
            'B-line': 6, 
            'I-line': 7,
            'B-v_ind': 8,
            'I-v_ind': 9,
            'O': 0}
    
id2label = {1: 'B-width',
            2: 'B-height', 
            3: 'B-radius', 
            4: 'I-radius',
            5: 'B-brand', 
            6: 'B-line', 
            7: 'I-line',
            8: 'B-v_ind',
            9: 'I-v_ind',
            0: 'O'}